In [155]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = file_names[0:2]
test_files = file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-155-4c6356020c63>", line 11, in <module>
    train_dataset = ProteinDataset(ids=train_files)
TypeError: __init__() missing 1 required positional argument: 'root'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1199, in str

In [156]:
class ProteinDataset(Dataset):
    def __init__(self, ids, transform=None, pre_transform=None):
        #super().__init__()
        ##file names equiv to ids
        super().__init__()
        # self.root = root
        self.raw_file_names = list(ids)
        self.processed_file_names = []
    # @property
    # def raw_file_names(self):

    #     return [filename for filename in os.scandir(self.root)]

    # @property
    # def processed_file_names(self):
    #     return [os.path.splitext(os.path.basename(file))[0]+'.pt' for file in self.raw_paths]

    def len(self):
        ## Can iter through len and load through batches
        return len(self.raw_file_names)

    
    def process(self):

        
        data_list = []
        count = 0
                
        for file in tqdm(self.raw_file_names):
        #    if (pathlib.Path(file).suffix == ".pdb"):
            struct = self._get_structure(file)
            # print (struct)
            seq = self._get_sequence(struct)
            #print(seq)
            #print(self._get_SeqVecEmbedder(seq))
            # node features extracted
            node_feats = self._get_one_hot_symbftrs(seq)
            # print(node_feats)
            # edge-index extracted
            mat = self._get_adjacency(file)
            #print(mat)
            
            edge_index = self._get_edgeindex(file, mat)
            #print(f'Node features size :{torch.Tensor.size(node_feats)}')
            #print(f'Matrix size :{mat.shape}')
            # create data object
            ### ADD Y = TARGET UB SITE AS binary fts (0./1)
            pos = ub_list.loc[ub_list['protein'] == self._get_id(file), 'site']
            #print(pos)
            ub_label = self._get_ubsite(seq, pos)
            #print(ub_label)
            data = Data(x=torch.tensor(node_feats, dtype=torch.float32),
                        y=torch.tensor(ub_label, dtype=torch.float32), 
                        edge_index=torch.tensor(edge_index, dtype=torch.long))
            #print(data)
            count += 1
            print(count)
            #print("asdf")
            data_list.append(data)
            self.processed_file_names.append(
                "./d/processed/" + f'data_{count}.pt')
            torch.save(data, "./d/processed/" + f'data_{count}.pt')
            
        return data_list


    def get(self, idx):
        
        return torch.load(f'./d/processed/data_{idx}.pt')
    
    
    
    def _get_id(self, pdb_file):
        pdb_id = pdb_file.split(".")[-2].split('/')[-1]
        #print(pdb_id)
        return pdb_id
    
    # get structure from a pdb file
    # Uses biopython
    def _get_structure(self, file):
        parser = PDBParser()
        structure = parser.get_structure(self._get_id(file), file)
        return structure

    # Function to get sequence from pdb structure
    # Uses structure made using biopython
    # Those residues for which symbols are U / X are converted into A

    
    def _get_sequence(self, structure):
        sequence = ""
        for model in structure:
          for chain in model:
            for residue in chain:
              if residue.get_resname() in ressymbl.keys():
                  sequence = sequence + ressymbl[residue.get_resname()]
        return sequence
        # One hot encoding for symbols

    def _get_ubsite(self,sequence, position):
        n = len(sequence)
        ub_array = np.zeros((n,))
        ub_array[position] = 1
        
        return ub_array

    def _get_one_hot_symbftrs(self, sequence):
        one_hot_symb = np.zeros((len(sequence), len(pro_res_table)))
        row = 0
        for res in sequence:
          col = pro_res_table.index(res)
          one_hot_symb[row][col] = 1
          row += 1
        return torch.tensor(one_hot_symb, dtype=torch.float)
    
    def _get_adjacency(self, file):
        
        ## Check how I can look into side chains vs AlphaChains in terms of close proximity (is it avg distance or min distance between two residues?)
        edge_ind =[]
        molecule = bg.Pmolecule(file)
        network = molecule.network()
        mat = nx.adjacency_matrix(network)
        m = mat.todense()
        return m
    # get adjacency matrix in coo format to pass in GCNN model

    def _get_edgeindex(self, file, adjacency_mat):
        edge_ind = []
        m = self._get_adjacency(file)
        # check_symmetric(m, rtol=1e-05, atol=1e-08)

        a = np.nonzero(m > 0)[0]
        b = np.nonzero(m > 0)[1]
        edge_ind.append(a)
        edge_ind.append(b)
        return torch.tensor(np.array(edge_ind), dtype=torch.long)
       # Residue features calculated from pcp_dict

    def _get_res_ftrs(self, sequence):
        res_ftrs_out = []
        for res in sequence:
          res_ftrs_out.append(pcp_dict[res])
        res_ftrs_out = np.array(res_ftrs_out)
        # print(res_ftrs_out.shape)
        return torch.tensor(res_ftrs_out, dtype=torch.float)
    
    # total features after concatenating one_hot_symbftrs and res_ftrs

    def _get_node_ftrs(self, sequence):
        one_hot_symb = one_hot_symbftrs(sequence)
        res_ftrs_out = res_ftrs(sequence)
        return torch.tensor(np.hstack((one_hot_symb, res_ftrs_out)), dtype=torch.float)

    def _get_SeqVecEmbedder(self, seq):

        embedder = SeqVecEmbedder()
        embedding = embedder.embed(seq)
        protein_embd = torch.tensor(embedding).sum(dim=0)  # Vector with shape [L x 1024]
        np_arr = protein_embd.cpu().detach().numpy()
        
        return np_arr
    
    def _get_ProtBertEmbedder(self, seq):

        embedder = ProtTransBertBFDEmbedder()
        embedding = embedder.embed(seq)
        protein_embd = torch.tensor(embedding).sum(dim=0)  # Vector with shape [L x 1024]
        np_arr = protein_embd.cpu().detach().numpy()
        
        return np_arr

# #%%    

In [157]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = file_names[0:2]
test_files = file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-157-4c6356020c63>", line 11, in <module>
    train_dataset = ProteinDataset(ids=train_files)
  File "<ipython-input-156-d53060bb82d1>", line 8, in __init__
    super().__init__()
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/data/dataset.py", line 115, in __init__
    self._process()
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/data/dataset.py", line 253, in _process
    if not self.force_reload and files_exist(self.processed_paths):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/data/dataset.py", line 212, in processed_paths
    files = self.processed_file_names
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib

In [158]:
?Dataset


Init signature:
Dataset(
    root: Optional[str] = None,
    transform: Optional[Callable] = None,
    pre_transform: Optional[Callable] = None,
    pre_filter: Optional[Callable] = None,
    log: bool = True,
    force_reload: bool = False,
) -> None
Docstring:     
Dataset base class for creating graph datasets.
See `here <https://pytorch-geometric.readthedocs.io/en/latest/tutorial/
create_dataset.html>`__ for the accompanying tutorial.

Args:
    root (str, optional): Root directory where the dataset should be saved.
        (optional: :obj:`None`)
    transform (callable, optional): A function/transform that takes in a
        :class:`~torch_geometric.data.Data` or
        :class:`~torch_geometric.data.HeteroData` object and returns a
        transformed version.
        The data object will be transformed before every access.
        (default: :obj:`None`)
    pre_transform (callable, optional): A function/transform that takes in
        a :class:`~torch_geometric.data.Data` or
  

In [160]:
class ProteinDataset(Dataset):
    def __init__(self, ids, transform=None, pre_transform=None):

        super().__init__()
        self.ids = ids
        
        
    def len(self):
        ## Can iter through len and load through batches
        return len(self.ids)

    def get(self, idx):
        
        return torch.load(f'./d/processed/data_{idx}.pt')
    

  
# #%%    

class PreProcessor():
    # init etc
    def process(self):

        data_list = []
        count = 0

        for file in tqdm(self.raw_file_names):
                #    if (pathlib.Path(file).suffix == ".pdb"):
                struct = self._get_structure(file)
                # print (struct)
                seq = self._get_sequence(struct)
                # print(seq)
                # print(self._get_SeqVecEmbedder(seq))
                # node features extracted
                node_feats = self._get_one_hot_symbftrs(seq)
                # print(node_feats)
                # edge-index extracted
                mat = self._get_adjacency(file)
                # print(mat)

                edge_index = self._get_edgeindex(file, mat)
                # print(f'Node features size :{torch.Tensor.size(node_feats)}')
                # print(f'Matrix size :{mat.shape}')
                # create data object
                # ADD Y = TARGET UB SITE AS binary fts (0./1)
                pos = ub_list.loc[ub_list['protein'] == self._get_id(file), 'site']
                # print(pos)
                ub_label = self._get_ubsite(seq, pos)
                # print(ub_label)
                data = Data(x=torch.tensor(node_feats, dtype=torch.float32),
                            y=torch.tensor(ub_label, dtype=torch.float32),
                            edge_index=torch.tensor(edge_index, dtype=torch.long))
                # print(data)
                count += 1
                print(count)
                # print("asdf")
                data_list.append(data)
                self.processed_file_names.append(
                    "./d/processed/" + f'data_{count}.pt')
                torch.save(data, "./d/processed/" + f'data_{count}.pt')

            return data_list

    # get structure from a pdb file
        # Uses biopython
        def _get_structure(self, file):
            parser = PDBParser()
            structure = parser.get_structure(self._get_id(file), file)
            return structure

        # Function to get sequence from pdb structure
        # Uses structure made using biopython
        # Those residues for which symbols are U / X are converted into A

        
        def _get_sequence(self, structure):
            sequence = ""
            for model in structure:
            for chain in model:
                for residue in chain:
                if residue.get_resname() in ressymbl.keys():
                    sequence = sequence + ressymbl[residue.get_resname()]
            return sequence
            # One hot encoding for symbols

        def _get_ubsite(self,sequence, position):
            n = len(sequence)
            ub_array = np.zeros((n,))
            ub_array[position] = 1
            
            return ub_array

        def _get_one_hot_symbftrs(self, sequence):
            one_hot_symb = np.zeros((len(sequence), len(pro_res_table)))
            row = 0
            for res in sequence:
            col = pro_res_table.index(res)
            one_hot_symb[row][col] = 1
            row += 1
            return torch.tensor(one_hot_symb, dtype=torch.float)
        
        def _get_adjacency(self, file):
            
            ## Check how I can look into side chains vs AlphaChains in terms of close proximity (is it avg distance or min distance between two residues?)
            edge_ind =[]
            molecule = bg.Pmolecule(file)
            network = molecule.network()
            mat = nx.adjacency_matrix(network)
            m = mat.todense()
            return m
        # get adjacency matrix in coo format to pass in GCNN model

        def _get_edgeindex(self, file, adjacency_mat):
            edge_ind = []
            m = self._get_adjacency(file)
            # check_symmetric(m, rtol=1e-05, atol=1e-08)

            a = np.nonzero(m > 0)[0]
            b = np.nonzero(m > 0)[1]
            edge_ind.append(a)
            edge_ind.append(b)
            return torch.tensor(np.array(edge_ind), dtype=torch.long)
        # Residue features calculated from pcp_dict

        def _get_res_ftrs(self, sequence):
            res_ftrs_out = []
            for res in sequence:
            res_ftrs_out.append(pcp_dict[res])
            res_ftrs_out = np.array(res_ftrs_out)
            # print(res_ftrs_out.shape)
            return torch.tensor(res_ftrs_out, dtype=torch.float)
        def _get_id(self, pdb_file):
            pdb_id = pdb_file.split(".")[-2].split('/')[-1]
            #print(pdb_id)
            return pdb_id
    
        # total features after concatenating one_hot_symbftrs and res_ftrs

        def _get_node_ftrs(self, sequence):
            one_hot_symb = one_hot_symbftrs(sequence)
            res_ftrs_out = res_ftrs(sequence)
            return torch.tensor(np.hstack((one_hot_symb, res_ftrs_out)), dtype=torch.float)
    
    def _get_id(self, pdb_file):
        pdb_id = pdb_file.split(".")[-2].split('/')[-1]
        #print(pdb_id)
        return pdb_id
    
        def _get_SeqVecEmbedder(self, seq):

            embedder = SeqVecEmbedder()
            embedding = embedder.embed(seq)
            protein_embd = torch.tensor(embedding).sum(dim=0)  # Vector with shape [L x 1024]
            np_arr = protein_embd.cpu().detach().numpy()
            
            return np_arr
        
        def _get_ProtBertEmbedder(self, seq):

            embedder = ProtTransBertBFDEmbedder()
            embedding = embedder.embed(seq)
            protein_embd = torch.tensor(embedding).sum(dim=0)  # Vector with shape [L x 1024]
            np_arr = protein_embd.cpu().detach().numpy()
            
            return np_arr

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 65)

In [161]:
class ProteinDataset(Dataset):
    def __init__(self, ids, transform=None, pre_transform=None):

        super().__init__()
        self.ids = ids
        
        
    def len(self):
        ## Can iter through len and load through batches
        return len(self.ids)

    def get(self, idx):
        
        return torch.load(f'./d/processed/data_{idx}.pt')

In [162]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = file_names[0:2]
test_files = file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

In [163]:
train_dataset

ProteinDataset(2)

In [164]:
[12:12 PM] Nathaniel Evans
train_dataset.get(0)

SyntaxError: invalid syntax (<ipython-input-164-36ebfda06ed7>, line 1)

In [165]:

train_dataset.get(0)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-165-e6355eaec6f9>", line 1, in <module>
    train_dataset.get(0)
  File "<ipython-input-161-d3afe8bae45f>", line 17, in get
    return torch.load(f'./d/processed/data_{idx}.pt')
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/serialization.py", line 594, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './

In [166]:

train_dataset.get(1)

Data(x=[92, 20], edge_index=[2, 856], y=[92])

In [167]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = ['./d/processed/data_1.pt', './d/processed/data_2.pt'] #file_names[0:2]
test_files = #file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

SyntaxError: invalid syntax (<ipython-input-167-a58cdd9dc808>, line 9)

In [168]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = ['./d/processed/data_1.pt', './d/processed/data_2.pt'] #file_names[0:2]
#test_files = #file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

In [169]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = ['./d/processed/data_1.pt', './d/processed/data_2.pt'] #file_names[0:2]
#test_files = #file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

In [170]:
class ProteinDataset(Dataset):
    def __init__(self, ids):

        super().__init__()
        self.ids = ids
        
    def len(self):
        ## Can iter through len and load through batches
        return len(self.ids)

    def get(self, idx):
        fname = self.ids[idx]
        return torch.load(fname)

In [171]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = ['./d/processed/data_1.pt', './d/processed/data_2.pt'] #file_names[0:2]
#test_files = #file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

In [172]:

train_dataset.get(0)


Data(x=[92, 20], edge_index=[2, 856], y=[92])

In [173]:

train_dataset.get(1)


Data(x=[92, 20], edge_index=[2, 856], y=[92])

In [174]:

train_dataset.get(2)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-174-7037c0f3111b>", line 1, in <module>
    train_dataset.get(2)
  File "<ipython-input-170-aa6ea365e0ed>", line 15, in get
    fname = self.ids[idx]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultrat

In [175]:
import os
import numpy as np
from torch.utils.data import Dataset as Dataset_n
seed = 42
torch.manual_seed(seed)
# print(math.floor(0.8 * size))
# Make iterables using dataloader class
#trainset, validset = torch.utils.data.random_split(
#    data_class, [2, 2])
# print(trainset[0])
trainloader = DataLoader(dataset=train_dataset, batch_size = 2,  num_workers=0)
#testloader = DataLoader(dataset=test_dataset, batch_size = 2, num_workers=0)
# print("Length")
# print(len(trainloader))
# print(len(testloader))

/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [176]:
next(iter(trainloader))

DataBatch(x=[184, 20], edge_index=[2, 1712], y=[184], batch=[184], ptr=[3])

In [177]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


def test():
    model.eval()
    out = model(trainloader.x, trainloader.edge_index)
    pred = out.argmax(dim=1)
    test_correct = pred[validloader.y]
    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-177-2bad1b57ce05>", line 28, in <module>
    loss = train()
  File "<ipython-input-177-2bad1b57ce05>", line 4, in train
    model.train()
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", li

In [178]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(trainloader.x, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, trainloader.y)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-178-0401f5adc4c6>", line 19, in <module>
    model = GCN(hidden_channels=16)
  File "<ipython-input-178-0401f5adc4c6>", line 8, in __init__
    self.conv1 = GCNConv(trainloader.x, hidden_channels)
AttributeError: 'DataLoader' object has no attribute 'x'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/op

In [179]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(trainloader.x, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, trainloader.y)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-179-496e547a32c0>", line 19, in <module>
    model = GCN(hidden_channels=16)
  File "<ipython-input-179-496e547a32c0>", line 8, in __init__
    self.conv1 = GCNConv(trainloader.x, hidden_channels)
AttributeError: 'DataLoader' object has no attribute 'x'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/op

In [180]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(trainloader.x, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, trainloader.y)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-180-496e547a32c0>", line 19, in <module>
    model = GCN(hidden_channels=16)
  File "<ipython-input-180-496e547a32c0>", line 8, in __init__
    self.conv1 = GCNConv(trainloader.x, hidden_channels)
AttributeError: 'DataLoader' object has no attribute 'x'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/op

In [181]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


def test():
    model.eval()
    out = model(trainloader.x, trainloader.edge_index)
    pred = out.argmax(dim=1)
    test_correct = pred[validloader.y]
    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-181-fb44d064f06a>", line 28, in <module>
    loss = train()
  File "<ipython-input-181-fb44d064f06a>", line 4, in train
    model.train()
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", li

In [182]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-182-cfda115620e4>", line 28, in <module>
    loss = train()
  File "<ipython-input-182-cfda115620e4>", line 4, in train
    model.train()
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", li

In [183]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(trainloader.x, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, trainloader.y)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-183-496e547a32c0>", line 19, in <module>
    model = GCN(hidden_channels=16)
  File "<ipython-input-183-496e547a32c0>", line 8, in __init__
    self.conv1 = GCNConv(trainloader.x, hidden_channels)
AttributeError: 'DataLoader' object has no attribute 'x'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/op

In [184]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-184-b50ee07e6fe4>", line 19, in <module>
    model = GCN(hidden_channels=16)
TypeError: __init__() missing 1 required positional argument: 'in_channels'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1199, in structured_t

In [185]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(in_channels = 20, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

In [186]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-186-cfda115620e4>", line 28, in <module>
    loss = train()
  File "<ipython-input-186-cfda115620e4>", line 8, in train
    out = model(data.x, data.edge_index)
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "<ipython-input-185-4d17933d627d>", line 12, in forward
    x = self.conv1(x, edge_index)
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/nn/conv/gcn_conv.py", line 241, in forward
    edge_index, edge_

In [188]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-188-cfda115620e4>", line 28, in <module>
    loss = train()
  File "<ipython-input-188-cfda115620e4>", line 8, in train
    out = model(data.x, data.edge_index)
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    def to(self: T, tensor: Tensor, non_blocking: bool = ...) -> T:
  File "<ipython-input-185-4d17933d627d>", line 12, in forward
    x = self.conv1(x, edge_index)
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    def to(self: T, tensor: Tensor, non_blocking: bool = ...) -> T:
  File "/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/nn/conv/gcn_

In [1]:
from torch_geometric.nn import GCNConv
from os.path import isfile, join
from os import listdir
import math
import random
from torch_geometric.data import DataLoader 
from bio_embeddings.embed import ProtTransBertBFDEmbedder, SeqVecEmbedder
from torch_geometric.data import Dataset, download_url, Data,  Batch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
import pathlib

import biographs as bg
from Bio import SeqIO
from Bio.PDB.PDBParser import PDBParser


import torch
import networkx as nx
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

import os.path as osp

/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ProteinDataset(Dataset):
    def __init__(self, ids):

        super().__init__()
        self.ids = ids
        
    def len(self):
        ## Can iter through len and load through batches
        return len(self.ids)

    def get(self, idx):
        fname = self.ids[idx]
        return torch.load(fname)

In [3]:
"""_summary_
## masking some of the nodes in the training sites

"""


train_files = ['./d/processed/data_1.pt', './d/processed/data_2.pt'] #file_names[0:2]
#test_files = #file_names[2:4]

train_dataset = ProteinDataset(ids=train_files)
# test_dataset = ProteinDataset(ids=test_files)

# train_dataset.process()
# test_dataset.process()
#data = data_class.process()

In [4]:
import os
import numpy as np
from torch.utils.data import Dataset as Dataset_n
seed = 42
torch.manual_seed(seed)
# print(math.floor(0.8 * size))
# Make iterables using dataloader class
#trainset, validset = torch.utils.data.random_split(
#    data_class, [2, 2])
# print(trainset[0])
trainloader = DataLoader(dataset=train_dataset, batch_size = 2,  num_workers=0)
#testloader = DataLoader(dataset=test_dataset, batch_size = 2, num_workers=0)
# print("Length")
# print(len(trainloader))
# print(len(testloader))

/Users/nguyjust/opt/anaconda3/envs/py9/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(in_channels = 20, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

In [6]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

ValueError: Using a target size (torch.Size([184])) that is different to the input size (torch.Size([184, 1])) is deprecated. Please ensure they have the same size.

In [7]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out, data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

ValueError: Using a target size (torch.Size([184])) that is different to the input size (torch.Size([184, 1])) is deprecated. Please ensure they have the same size.

In [8]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

RuntimeError: all elements of input should be between 0 and 1

In [9]:
data.y

NameError: name 'data' is not defined

In [10]:
data = next(iter(trainloader))

In [11]:


data.y

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)


model = GCN(in_channels = 20, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

In [13]:
def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 30):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.6847
Epoch: 002, Loss: 0.6619
Epoch: 003, Loss: 0.6310
Epoch: 004, Loss: 0.6092
Epoch: 005, Loss: 0.5783
Epoch: 006, Loss: 0.5484
Epoch: 007, Loss: 0.5128
Epoch: 008, Loss: 0.4884
Epoch: 009, Loss: 0.4509
Epoch: 010, Loss: 0.4382
Epoch: 011, Loss: 0.4030
Epoch: 012, Loss: 0.3748
Epoch: 013, Loss: 0.3577
Epoch: 014, Loss: 0.3323
Epoch: 015, Loss: 0.3039
Epoch: 016, Loss: 0.2761
Epoch: 017, Loss: 0.2393
Epoch: 018, Loss: 0.2317
Epoch: 019, Loss: 0.2230
Epoch: 020, Loss: 0.1933
Epoch: 021, Loss: 0.1897
Epoch: 022, Loss: 0.1702
Epoch: 023, Loss: 0.1571
Epoch: 024, Loss: 0.1388
Epoch: 025, Loss: 0.1258
Epoch: 026, Loss: 0.1258
Epoch: 027, Loss: 0.1126
Epoch: 028, Loss: 0.1126
Epoch: 029, Loss: 0.1102


In [14]:
model(data.x, data.edge_index)

tensor([[0.1229],
        [0.0826],
        [0.0554],
        [0.0605],
        [0.0495],
        [0.0503],
        [0.0348],
        [0.0496],
        [0.0231],
        [0.0583],
        [0.0418],
        [0.0430],
        [0.0562],
        [0.0617],
        [0.0803],
        [0.0559],
        [0.0488],
        [0.0500],
        [0.0427],
        [0.0319],
        [0.0302],
        [0.0407],
        [0.0331],
        [0.0378],
        [0.0466],
        [0.0515],
        [0.0437],
        [0.0627],
        [0.0824],
        [0.0567],
        [0.0974],
        [0.0818],
        [0.0860],
        [0.1051],
        [0.0813],
        [0.0878],
        [0.0588],
        [0.0304],
        [0.0667],
        [0.0482],
        [0.0491],
        [0.0666],
        [0.0854],
        [0.0776],
        [0.0884],
        [0.0730],
        [0.1343],
        [0.1106],
        [0.0536],
        [0.0591],
        [0.0930],
        [0.0621],
        [0.0531],
        [0.0413],
        [0.0992],
        [0

In [15]:
model(data.x, data.edge_index).max()

tensor(0.1628, grad_fn=<MaxBackward1>)

In [16]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = crit(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.0984
Epoch: 002, Loss: 0.0934
Epoch: 003, Loss: 0.0825
Epoch: 004, Loss: 0.0812
Epoch: 005, Loss: 0.0778
Epoch: 006, Loss: 0.0813
Epoch: 007, Loss: 0.0739
Epoch: 008, Loss: 0.0752
Epoch: 009, Loss: 0.0725
Epoch: 010, Loss: 0.0713
Epoch: 011, Loss: 0.0747
Epoch: 012, Loss: 0.0637
Epoch: 013, Loss: 0.0669
Epoch: 014, Loss: 0.0724
Epoch: 015, Loss: 0.0652
Epoch: 016, Loss: 0.0607
Epoch: 017, Loss: 0.0629
Epoch: 018, Loss: 0.0667
Epoch: 019, Loss: 0.0604
Epoch: 020, Loss: 0.0615
Epoch: 021, Loss: 0.0650
Epoch: 022, Loss: 0.0628
Epoch: 023, Loss: 0.0599
Epoch: 024, Loss: 0.0591
Epoch: 025, Loss: 0.0651
Epoch: 026, Loss: 0.0606
Epoch: 027, Loss: 0.0659
Epoch: 028, Loss: 0.0568
Epoch: 029, Loss: 0.0573
Epoch: 030, Loss: 0.0595
Epoch: 031, Loss: 0.0643
Epoch: 032, Loss: 0.0584
Epoch: 033, Loss: 0.0643
Epoch: 034, Loss: 0.0567
Epoch: 035, Loss: 0.0582
Epoch: 036, Loss: 0.0653
Epoch: 037, Loss: 0.0688
Epoch: 038, Loss: 0.0577
Epoch: 039, Loss: 0.0614
Epoch: 040, Loss: 0.0639


In [17]:

model(data.x, data.edge_index).max()

tensor(0.0447, grad_fn=<MaxBackward1>)

In [18]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        pos_idx = (data.y == 1).nonzero(as_tuple=True)[0]
        neg_idx = (data.y == 0).nonzero(as_tuple=True)[0]
        neg_idx = neg_idx[torch.randint(0, len(neg_idx), size=10)]
        idx = torch.cat((pos_idx, neg_idx), dim=-1)
        loss = crit(out.squeeze()[idx], data.y[idx])
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

TypeError: randint() received an invalid combination of arguments - got (int, int, size=int), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [19]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        pos_idx = (data.y == 1).nonzero(as_tuple=True)[0]
        neg_idx = (data.y == 0).nonzero(as_tuple=True)[0]
        neg_idx = neg_idx[torch.randint(0, len(neg_idx), size=(10,))]
        idx = torch.cat((pos_idx, neg_idx), dim=-1)
        loss = crit(out.squeeze()[idx], data.y[idx])
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.7899
Epoch: 002, Loss: 0.6791
Epoch: 003, Loss: 0.6690
Epoch: 004, Loss: 0.6525
Epoch: 005, Loss: 0.6449
Epoch: 006, Loss: 0.6410
Epoch: 007, Loss: 0.5853
Epoch: 008, Loss: 0.5589
Epoch: 009, Loss: 0.5651
Epoch: 010, Loss: 0.5275
Epoch: 011, Loss: 0.5164
Epoch: 012, Loss: 0.4913
Epoch: 013, Loss: 0.4141
Epoch: 014, Loss: 0.4054
Epoch: 015, Loss: 0.4097
Epoch: 016, Loss: 0.4278
Epoch: 017, Loss: 0.3987
Epoch: 018, Loss: 0.3477
Epoch: 019, Loss: 0.3515
Epoch: 020, Loss: 0.3721
Epoch: 021, Loss: 0.3600
Epoch: 022, Loss: 0.3817
Epoch: 023, Loss: 0.4124
Epoch: 024, Loss: 0.3734
Epoch: 025, Loss: 0.3344
Epoch: 026, Loss: 0.3589
Epoch: 027, Loss: 0.3678
Epoch: 028, Loss: 0.3527
Epoch: 029, Loss: 0.3904
Epoch: 030, Loss: 0.3864
Epoch: 031, Loss: 0.3716
Epoch: 032, Loss: 0.4155
Epoch: 033, Loss: 0.3572
Epoch: 034, Loss: 0.3628
Epoch: 035, Loss: 0.3764
Epoch: 036, Loss: 0.3331
Epoch: 037, Loss: 0.3360
Epoch: 038, Loss: 0.3424
Epoch: 039, Loss: 0.3235
Epoch: 040, Loss: 0.3134


In [20]:

model(data.x, data.edge_index).max()

tensor(0.4729, grad_fn=<MaxBackward1>)

In [21]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        pos_idx = (data.y == 1).nonzero(as_tuple=True)[0]
        neg_idx = (data.y == 0).nonzero(as_tuple=True)[0]
        neg_idx = neg_idx[torch.randint(0, len(neg_idx), size=(1,))]
        idx = torch.cat((pos_idx, neg_idx), dim=-1)
        loss = crit(out.squeeze()[idx], data.y[idx])
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.7554
Epoch: 002, Loss: 0.7185
Epoch: 003, Loss: 0.6658
Epoch: 004, Loss: 0.7289
Epoch: 005, Loss: 0.7822
Epoch: 006, Loss: 0.6432
Epoch: 007, Loss: 0.7016
Epoch: 008, Loss: 0.5442
Epoch: 009, Loss: 0.6866
Epoch: 010, Loss: 0.5204
Epoch: 011, Loss: 0.5414
Epoch: 012, Loss: 0.5296
Epoch: 013, Loss: 0.4783
Epoch: 014, Loss: 0.6756
Epoch: 015, Loss: 0.4889
Epoch: 016, Loss: 0.6406
Epoch: 017, Loss: 0.4898
Epoch: 018, Loss: 0.5882
Epoch: 019, Loss: 0.5964
Epoch: 020, Loss: 0.4541
Epoch: 021, Loss: 0.4252
Epoch: 022, Loss: 0.5611
Epoch: 023, Loss: 0.4414
Epoch: 024, Loss: 0.4380
Epoch: 025, Loss: 0.4644
Epoch: 026, Loss: 0.4319
Epoch: 027, Loss: 0.5579
Epoch: 028, Loss: 0.4751
Epoch: 029, Loss: 0.4279
Epoch: 030, Loss: 0.4141
Epoch: 031, Loss: 0.3931
Epoch: 032, Loss: 0.4367
Epoch: 033, Loss: 0.4029
Epoch: 034, Loss: 0.3700
Epoch: 035, Loss: 0.4602
Epoch: 036, Loss: 0.4139
Epoch: 037, Loss: 0.3438
Epoch: 038, Loss: 0.3620
Epoch: 039, Loss: 0.3777
Epoch: 040, Loss: 0.4011


In [22]:

model(data.x, data.edge_index).max()

tensor(0.7556, grad_fn=<MaxBackward1>)

In [23]:

(data.y == (model(data.x, data.edge_index) > 0.5)).mean()

RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating point or complex dtype. Got: Bool

In [24]:

(1.*(data.y == (model(data.x, data.edge_index) > 0.5))).mean()

tensor(0.8509)

In [25]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)


model = GCN(in_channels = 20, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

In [26]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        pos_idx = (data.y == 1).nonzero(as_tuple=True)[0]
        neg_idx = (data.y == 0).nonzero(as_tuple=True)[0]
        neg_idx = neg_idx[torch.randint(0, len(neg_idx), size=(1,))]
        idx = torch.cat((pos_idx, neg_idx), dim=-1)
        loss = crit(out.squeeze()[idx], data.y[idx])
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.7009
Epoch: 002, Loss: 0.6985
Epoch: 003, Loss: 0.6745
Epoch: 004, Loss: 0.6614
Epoch: 005, Loss: 0.6610
Epoch: 006, Loss: 0.6562
Epoch: 007, Loss: 0.6536
Epoch: 008, Loss: 0.6339
Epoch: 009, Loss: 0.6467
Epoch: 010, Loss: 0.6355
Epoch: 011, Loss: 0.5954
Epoch: 012, Loss: 0.6197
Epoch: 013, Loss: 0.6335
Epoch: 014, Loss: 0.6006
Epoch: 015, Loss: 0.5888
Epoch: 016, Loss: 0.6549
Epoch: 017, Loss: 0.5530
Epoch: 018, Loss: 0.5711
Epoch: 019, Loss: 0.5872
Epoch: 020, Loss: 0.5938
Epoch: 021, Loss: 0.5420
Epoch: 022, Loss: 0.5125
Epoch: 023, Loss: 0.6516
Epoch: 024, Loss: 0.5626
Epoch: 025, Loss: 0.5583
Epoch: 026, Loss: 0.5312
Epoch: 027, Loss: 0.6478
Epoch: 028, Loss: 0.5505
Epoch: 029, Loss: 0.5123
Epoch: 030, Loss: 0.5208
Epoch: 031, Loss: 0.5391
Epoch: 032, Loss: 0.5368
Epoch: 033, Loss: 0.5449
Epoch: 034, Loss: 0.4462
Epoch: 035, Loss: 0.4867
Epoch: 036, Loss: 0.5392
Epoch: 037, Loss: 0.6629
Epoch: 038, Loss: 0.4791
Epoch: 039, Loss: 0.4647
Epoch: 040, Loss: 0.4449


In [27]:


(1.*(data.y == (model(data.x, data.edge_index) > 0.5))).mean()

tensor(0.7871)

In [28]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)


model = GCN(in_channels = 20, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)
#criterion = torch.nn.CrossEntropyLoss()
crit = torch.nn.BCELoss() # target is between 0,1, yhat is unnormalized logits

In [29]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        pos_idx = (data.y == 1).nonzero(as_tuple=True)[0]
        neg_idx = (data.y == 0).nonzero(as_tuple=True)[0]
        neg_idx = neg_idx[torch.randint(0, len(neg_idx), size=(1,))]
        idx = torch.cat((pos_idx, neg_idx), dim=-1)
        loss = crit(out.squeeze()[idx], data.y[idx])
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.7009
Epoch: 002, Loss: 0.7073
Epoch: 003, Loss: 0.6952
Epoch: 004, Loss: 0.6920
Epoch: 005, Loss: 0.7009
Epoch: 006, Loss: 0.7030
Epoch: 007, Loss: 0.6925
Epoch: 008, Loss: 0.6969
Epoch: 009, Loss: 0.6907
Epoch: 010, Loss: 0.6926
Epoch: 011, Loss: 0.6928
Epoch: 012, Loss: 0.6952
Epoch: 013, Loss: 0.6872
Epoch: 014, Loss: 0.6919
Epoch: 015, Loss: 0.6864
Epoch: 016, Loss: 0.6897
Epoch: 017, Loss: 0.6856
Epoch: 018, Loss: 0.6801
Epoch: 019, Loss: 0.6867
Epoch: 020, Loss: 0.6924
Epoch: 021, Loss: 0.6784
Epoch: 022, Loss: 0.6685
Epoch: 023, Loss: 0.6841
Epoch: 024, Loss: 0.6852
Epoch: 025, Loss: 0.6807
Epoch: 026, Loss: 0.6786
Epoch: 027, Loss: 0.6807
Epoch: 028, Loss: 0.6757
Epoch: 029, Loss: 0.6737
Epoch: 030, Loss: 0.6788
Epoch: 031, Loss: 0.6772
Epoch: 032, Loss: 0.6709
Epoch: 033, Loss: 0.6740
Epoch: 034, Loss: 0.6535
Epoch: 035, Loss: 0.6724
Epoch: 036, Loss: 0.6658
Epoch: 037, Loss: 0.6683
Epoch: 038, Loss: 0.6604
Epoch: 039, Loss: 0.6714
Epoch: 040, Loss: 0.6614


In [30]:



(1.*(data.y == (model(data.x, data.edge_index) > 0.5))).mean()

tensor(0.0109)

In [31]:
model(data.x, data.edge_index)

tensor([[0.5344],
        [0.5386],
        [0.5510],
        [0.5554],
        [0.5786],
        [0.5887],
        [0.6054],
        [0.6034],
        [0.6224],
        [0.6124],
        [0.6373],
        [0.6382],
        [0.6338],
        [0.6345],
        [0.6257],
        [0.6046],
        [0.5965],
        [0.5783],
        [0.5827],
        [0.6027],
        [0.5905],
        [0.5816],
        [0.5903],
        [0.5903],
        [0.5788],
        [0.5767],
        [0.5830],
        [0.5677],
        [0.5521],
        [0.5672],
        [0.5632],
        [0.5798],
        [0.5733],
        [0.5786],
        [0.5801],
        [0.5812],
        [0.5870],
        [0.5959],
        [0.5708],
        [0.5724],
        [0.5553],
        [0.5547],
        [0.5430],
        [0.5668],
        [0.5661],
        [0.5777],
        [0.5735],
        [0.5763],
        [0.5930],
        [0.5999],
        [0.5897],
        [0.5965],
        [0.6069],
        [0.6106],
        [0.5876],
        [0

In [32]:
from sklearn.metrics import roc_auc_score

def train():
    
    model.train()
    losses = 0
    for batch_idx, data in enumerate(trainloader): 
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        pos_idx = (data.y == 1).nonzero(as_tuple=True)[0]
        neg_idx = (data.y == 0).nonzero(as_tuple=True)[0]
        neg_idx = neg_idx[torch.randint(0, len(neg_idx), size=(10,))]
        idx = torch.cat((pos_idx, neg_idx), dim=-1)
        loss = crit(out.squeeze()[idx], data.y[idx])
        loss.backward()
        optimizer.step()
        losses += loss.item() # .detach 

    return loss


#def test():
#    model.eval()
#    out = model(trainloader.x, trainloader.edge_index)
#    pred = out.argmax(dim=1)
#    test_correct = pred[testloader.y]
#    #test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
#    return test_correct 



for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 0.7880
Epoch: 002, Loss: 0.7892
Epoch: 003, Loss: 0.8299
Epoch: 004, Loss: 0.8146
Epoch: 005, Loss: 0.7964
Epoch: 006, Loss: 0.8216
Epoch: 007, Loss: 0.7842
Epoch: 008, Loss: 0.7921
Epoch: 009, Loss: 0.7836
Epoch: 010, Loss: 0.7619
Epoch: 011, Loss: 0.7765
Epoch: 012, Loss: 0.7948
Epoch: 013, Loss: 0.7475
Epoch: 014, Loss: 0.7773
Epoch: 015, Loss: 0.7822
Epoch: 016, Loss: 0.7391
Epoch: 017, Loss: 0.7629
Epoch: 018, Loss: 0.7569
Epoch: 019, Loss: 0.7227
Epoch: 020, Loss: 0.7451
Epoch: 021, Loss: 0.7337
Epoch: 022, Loss: 0.7341
Epoch: 023, Loss: 0.7361
Epoch: 024, Loss: 0.7092
Epoch: 025, Loss: 0.7219
Epoch: 026, Loss: 0.7372
Epoch: 027, Loss: 0.7189
Epoch: 028, Loss: 0.6954
Epoch: 029, Loss: 0.7114
Epoch: 030, Loss: 0.6814
Epoch: 031, Loss: 0.6844
Epoch: 032, Loss: 0.7056
Epoch: 033, Loss: 0.7054
Epoch: 034, Loss: 0.6728
Epoch: 035, Loss: 0.6659
Epoch: 036, Loss: 0.6640
Epoch: 037, Loss: 0.6715
Epoch: 038, Loss: 0.6585
Epoch: 039, Loss: 0.6620
Epoch: 040, Loss: 0.6678


In [33]:

model(data.x, data.edge_index)

tensor([[0.3438],
        [0.3092],
        [0.2694],
        [0.2921],
        [0.2680],
        [0.3017],
        [0.2923],
        [0.3408],
        [0.2846],
        [0.3686],
        [0.3434],
        [0.3825],
        [0.3914],
        [0.3856],
        [0.3821],
        [0.3372],
        [0.3104],
        [0.3133],
        [0.3069],
        [0.2887],
        [0.2890],
        [0.3041],
        [0.2913],
        [0.2957],
        [0.3235],
        [0.3192],
        [0.2843],
        [0.3221],
        [0.3318],
        [0.3134],
        [0.3322],
        [0.3224],
        [0.3286],
        [0.3435],
        [0.3326],
        [0.3387],
        [0.3054],
        [0.2720],
        [0.2988],
        [0.2907],
        [0.2887],
        [0.2947],
        [0.3078],
        [0.2968],
        [0.3014],
        [0.2989],
        [0.3417],
        [0.3224],
        [0.2823],
        [0.3045],
        [0.3328],
        [0.3048],
        [0.3043],
        [0.3202],
        [0.3479],
        [0

In [34]:



(1.*(data.y == (model(data.x, data.edge_index) > 0.5))).mean()

tensor(0.9891)